# Concrete Pad Foundation Check

In [213]:
import math

In [214]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global dictionary to store widget values (persists user entries)
try:
    widget_values
except NameError:
    widget_values = {}

# Define the form data based on the provided text
form_data = [
    {"name": "column_depth", "value": 0.400, "unit": "[m], depth of column", "type": "float"},
    {"name": "pad_width_x", "value": 1.2, "unit": "[m], horizontal dimension of concrete pad (cross)", "type": "float"},
    {"name": "pad_width_y", "value": 1.2, "unit": "[m], horizontal dimension of concrete pad (longitudinal)", "type": "float"},
    {"name": "pad_depth", "value": 1.5, "unit": "[m], depth of pad (ignoring slab)", "type": "float"},
    {"name": "dist_from_edge_x_direction", "value": 0.4, "unit": "[m], distance of column axis from the edge", "type": "float"},
    {"name": "dist_from_edge_y_direction", "value": 0.4, "unit": "[m], distance of column axis from the edge", "type": "float"},
    {"name": "slab_engaged_rotation", "value": True, "unit": "", "type": "bool"},
    {"name": "slab_engaged_uplift", "value": True, "unit": "", "type": "bool"},
    {"name": "Rz_down", "value": 50.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "Rz_uplift", "value": 40.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "Rx_plus", "value": 30.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "Rx_minus", "value": 30.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "Ry_plus_lngtnl_bracing", "value": 35.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "Ry_minus_lngtnl_bracing", "value": 35.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "Rz_uplift_bracing", "value": 20.0, "unit": "[kN] from frame analysis", "type": "float"},
    {"name": "My_plus", "value": 50.0, "unit": "[kNm] from frame analysis", "type": "float"},
    {"name": "My_minus", "value": 0.00, "unit": "[kNm] from frame analysis", "type": "float"},
    {"name": "Mx_plus", "value": 0.00, "unit": "[kNm] from frame analysis", "type": "float"},
    {"name": "Mx_minus", "value": 0.00, "unit": "[kNm] from frame analysis", "type": "float"},
    {"name": "gamma_1", "value": 20.0, "unit": "[kN/m^2]", "type": "float"},
    {"name": "gamma_2", "value": 20.0, "unit": "[kN/m^2]", "type": "float"},
    {"name": "phi_deg", "value": 18.0, "unit": "[deg]", "type": "float"},
    {"name": "c", "value": 18.00, "unit": "", "type": "float"},
    {"name": "c_coeff", "value": 2.0, "unit": "", "type": "float"},
    {"name": "delta_deg", "value": 0, "unit": "[deg]", "type": "float"},
    {"name": "frame_span", "value": 14.0, "unit": "[m]", "type": "float"},
    {"name": "frame_spacing", "value": 5.0, "unit": "[m]", "type": "float"},
    {"name": "slab_percentage", "value": 0.7, "unit": "", "type": "float"},
    {"name": "slab_thickness", "value": 0.15, "unit": "[m]", "type": "float"},
    {"name": "slab_reinf_dist_top", "value": 0.05, "unit": "[m], axial distance of reinforcement from the top of the slab", "type": "float"},
    {"name": "slab_reinf_dist_btm", "value": 0.05, "unit": "[m], axial distance of reinforcement from the btm of the slab", "type": "float"},
    {"name": "slab_reinforcement_area_top", "value": 786.0, "unit": "[mm^2/m]", "type": "int"},
    {"name": "slab_reinforcement_area_btm", "value": 786.0, "unit": "[mm^2/m]", "type": "int"},
    {"name": "slab_reinforcement_fy", "value": 355000.0, "unit": "[kPa]", "type": "float"},
    {"name": "slab_concrete_grade", "value": 25000.0, "unit": "[kPa]", "type": "float"},
    {"name": "concrete_density", "value": 24.0, "unit": "", "type": "float"},
]

# Create widgets for each variable
widgets_dict = {}
output = widgets.Output()

# Function to sync widget values to global variables, store values, and validate
def sync_globals(change):
    widget_name = None
    for item in form_data:
        if widgets_dict[item["name"]] is change["owner"]:
            widget_name = item["name"]
            break
    
    if widget_name:
        widget = widgets_dict[widget_name]
        value = widget.value
        widget_values[widget_name] = value  # Store user-entered value
        
        with output:
            clear_output()
            errors = []
            try:
                if item["type"] != "bool" and value < 0:
                    errors.append(f"{widget_name}: Value cannot be negative")
                globals()[widget_name] = value
            except (ValueError, TypeError):
                errors.append(f"{widget_name}: Invalid non-numeric input")
            
            if errors:
                print("Errors found:")
                for error in errors:
                    print(f"- {error}")

# Create widgets and set up observers
for item in form_data:
    label = widgets.Label(value=item["name"])
    # Use stored value if available, else default
    initial_value = widget_values.get(item["name"], item["value"])
    
    if item["type"] == "bool":
        widget = widgets.Checkbox(value=initial_value, description="", indent=False)
    elif item["type"] == "int":
        widget = widgets.IntText(value=int(initial_value), description="", layout={"width": "100px"})
    else:  # float
        widget = widgets.FloatText(value=float(initial_value), description="", layout={"width": "100px"})
    
    unit = widgets.Label(value=item["unit"])
    widgets_dict[item["name"]] = widget
    item["row"] = widgets.HBox([label, widget, unit], layout={"margin": "5px"})
    
    # Set initial global variable
    globals()[item["name"]] = widget.value
    widget_values[item["name"]] = widget.value  # Store initial value
    
    # Observe changes to sync global variables and validate
    widget.observe(sync_globals, names="value")

# Organize widgets into sections
sections = {
    "Column and Pad Dimensions": [
        "column_depth", "pad_width_x", "pad_width_y", "pad_depth",
        "dist_from_edge_x_direction", "dist_from_edge_y_direction",
        "slab_engaged_rotation", "slab_engaged_uplift"
    ],
    "Reactions": [
        "Rz_down", "Rz_uplift", "Rx_plus", "Rx_minus",
        "Ry_plus_lngtnl_bracing", "Ry_minus_lngtnl_bracing",
        "Rz_uplift_bracing", "My_plus", "My_minus", "Mx_plus", "Mx_minus"
    ],
    "Soil Properties": [
        "gamma_1", "gamma_2", "phi_deg", "c", "c_coeff", "delta_deg"
    ],
    "Frame Dimensions": [
        "frame_span", "frame_spacing"
    ],
    "Concrete Slab Properties": [
        "slab_percentage", "slab_thickness", "slab_reinf_dist_top",
        "slab_reinf_dist_btm", "slab_reinforcement_area_top",
        "slab_reinforcement_area_btm", "slab_reinforcement_fy",
        "slab_concrete_grade", "concrete_density"
    ]
}

# Create sectioned layout
section_widgets = []
for section_name, var_names in sections.items():
    section_label = widgets.HTML(value=f"<h3>{section_name}</h3>")
    section_rows = [next(item["row"] for item in form_data if item["name"] == var_name) for var_name in var_names]
    section_widgets.append(widgets.VBox([section_label] + section_rows))

# Display the form and output
form = widgets.VBox(section_widgets + [output])
display(form)

In [215]:
# Point load to be specified within the marked area (below, quadrant [1]) or on it's borders.
# Loading in [1] quadrant only,
# Loading specified in absolute values but considering direction,
# Loding CANNOT be specified in quadrants [2], [3] and [4],
# RIGHT HAND RULE APPLIES:
# x-axis is in the direction of the frame.
# y-axis is in the longitudinal direction
# z-axis is vertical, pointing up
# Calculates stresses at [1] and [2] ([3] and [4] are not critical)
# [4] ------^ +y-------[3]
# ǀ         ǀ           ǀ
# ǀ         ǀ[0,0]      ǀ
#-----------x---------------> +x  +z pointing upwards
# ǀ         ǀ           ǀ    
# ǀ  x(Load)ǀ           ǀ
# [1]-------ǀ----------[2] 

##column_depth = 0.400         #[m], depth of column
##pad_width_x = 1.2           #[m], horizontal dimension of concrete pad (cross)
##pad_width_y = 1.2           #[m], horizontal dimension of concrete pad (longitudinal)
##pad_depth = 1.5            #[m], depth of pad (ignoring slab)

##dist_from_edge_x_direction = 0.4     #[m], distance of column axis from the edge
##dist_from_edge_y_direction = 0.4      #[m], distance of column axis from the edge

##slab_engaged_rotation = True
##slab_engaged_uplift = True

# Reactions
##Rz_down = 48.41                    #[kN]  from LGSF  
##Rz_uplift = 39.39                  #[kN]  from LGSF
##Rx_plus = 30.17                    #[kN]  from LGSF
##Rx_minus = 30.17                   #[kN]  from LGSF
##Ry_plus_lngtnl_bracing = 35.0      #[kN]  not calculated here  
##Ry_minus_lngtnl_bracing = 35.0     #[kN]  not calculated here  
##Rz_uplift_bracing = 20.0           #[kN]  not calculated here
##My_plus = 50.0                     #[kNm] from LGSF
##My_minus = 0.00                    #[kNm] from LGSF
##Mx_plus = 0.00                     #[kNm] either zero or very small number in case of rigid bases
##Mx_minus = 0.00                    #[kNm] either zero or very small number in case of rigid bases

# Net allowable bearing pressure
##p = 80  #[kPa]

# Density of soil
##gamma_1 = 20.0            #[kN/m^2]
##gamma_2 = 20.0            #[kN/m^2]

# friction angle
##phi_deg = 18.0            #[deg]
##phi_rad = phi_deg * math.pi/180

# coefficient of cohesiveness
##c = 18.00

# coefficient for c variation
##c_coeff = 2.0

# load angle from vertical
##delta_deg = 0                             #[deg]
##delta_rad = delta_deg * math.pi/180

# frame dimansions
##frame_span = 14.0                         #[m]
##frame_spacing = 5.0                       #[m]

# concrete slab properties
# Assuming mesh reinforcement

##slab_percentage = 0.7
##slab_thickness = 0.15                     #[m]
##slab_reinf_dist_top = 0.05                #[m], axial distance of reinforcement from the top of the slab
##slab_reinf_dist_btm = 0.05                #[m], axial distance of reinforcement from the btm of the slab
##slab_reinforcement_area_top = 786.0       #[mm^2/m] 
##slab_reinforcement_area_btm = 786.0       #[mm^2/m]
##slab_reinforcement_fy = 355000.0          #[kPa]
##slab_concrete_grade = 25000.0             #[kPa]
##concrete_density = 24.0                   #[kN/m^2]

In [216]:
# Revise units
slab_reinforcement_area_top = slab_reinforcement_area_top / 1000000       #[m^2/m] 
slab_reinforcement_area_btm = slab_reinforcement_area_btm / 1000000       #[m^2/m]

### Check data

In [217]:
# Check data
# Conservative approach (input data reduction) of dealing with sign convention of forces and load eccentricities.
# Put all in one sense, possibly too conservative for some foundations.
fail_data = False
if dist_from_edge_x_direction > pad_width_x/2:
    fail_data= True
    print("Error in data, eccentricity in x direction is grater then half pad width in x direction.")    
if dist_from_edge_y_direction > pad_width_y/2:
    fail_data= True
    print("Error in data, eccentricity in x direction is grater then half pad width in x direction.")
    
if column_depth > pad_width_x:
    fail_data= True
    print("Error in data, column depth is greater then the footing width.")
    
if pad_depth < 0.8:
    fail_data= True
    print("Foundation depth is less then 80 cm freezing depth.")
    
if pad_depth > 3.0:
    fail_data= True
    print("Error in data. The depth of foundation seems to be too high.")
    
if pad_width_x < 0.3:
    print("Error in data. Width of foundation too low.")

if pad_width_y < 0.3:
    print("Error in data. Width of foundation too low.")

if pad_width_x > 2.0:
    print("Error in data. Width of foundation too high.")

if pad_width_y > 2.0:
    print("Error in data. Width of foundation too high.")
    
if slab_thickness < 0.1:
    print("Error in data. Slab thickness is too low.")

if slab_thickness > 0.5:
    print("Error in data. Slab thickness is too high.")
    
if slab_reinf_dist_top < 0.02:
    print("Error in data. Distance of reinforcement from surface seems to be too low. Check also minimum cover.")
    
if slab_reinf_dist_btm < 0.02:
    print("Error in data. Distance of reinforcement from surface seems to be too low. Check also minimum cover.")
    
if fail_data == False:
    print("Data OK")

Data OK


### Soil parameters

In [218]:
cd = c / c_coeff

# area of foundation
A = pad_width_x * pad_width_y


Nd = (math.tan(45 * math.pi/180.0 + 0.5*phi_rad)**(2)) * (math.exp(math.pi * math.tan(phi_rad)))
Nc = (Nd - 1) / math.tan(phi_rad)
Nb = 1.5 * (Nd - 1) * math.tan(phi_rad)

b = pad_width_x     # x direction
l = pad_width_y      # y direction
d = pad_depth

# coefficient of pad foundation dimensions
sc = 1 + 0.2 * b/l
sd = 1 + b/l * math.sin(phi_rad)
sb = 1 - 0.3 * b/l

# coefficient of pad foundation depth
dc = 1 + 0.1 * math.sqrt(d/b)
dd = 1 + 0.1 * math.sqrt(d/b * math.sin(2 * phi_rad))
db = 1.0

# coefficient for inclined load direction
ic = (1 - math.tan(delta_rad))**(2)
id = ic
ib = ic

# Design strength of soil
Rd = cd * Nc * sc * dc * ic + gamma_1 * d * Nd * sd * dd * id + gamma_2 * 0.5 * b * Nb * sb * db * ib

In [219]:
print("cd                                = ", ("%2.2f" % cd), " [-]")
print("ic                                = ", ("%2.2f" % ic), " [-]")
print("id                                = ", ("%2.2f" % id), " [-]")
print("ib                                = ", ("%2.2f" % ib), " [-]")
print("Nc                                = ", ("%2.2f" % Nc), " [-]")
print("Nd                                = ", ("%2.2f" % Nd), " [-]")
print("Nb                                = ", ("%2.2f" % Nb), " [-]")
print("sc                                = ", ("%2.2f" % sc), " [-]")
print("sd                                = ", ("%2.2f" % sd), " [-]")
print("sb                                = ", ("%2.2f" % sb), " [-]")
print("Rd                                = ", ("%2.2f" % Rd), " [kPa]")

cd                                =  9.00  [-]
ic                                =  1.00  [-]
id                                =  1.00  [-]
ib                                =  1.00  [-]
Nc                                =  13.10  [-]
Nd                                =  5.26  [-]
Nb                                =  2.08  [-]
sc                                =  1.20  [-]
sd                                =  1.31  [-]
sb                                =  0.70  [-]
Rd                                =  318.40  [kPa]


### Check stress at contact

In [220]:
weight_pad = concrete_density * pad_width_x * pad_width_y * pad_depth
sigma_pad = weight_pad / (pad_width_x * pad_width_y)

weight_slab = 0
if slab_engaged_uplift:
    weight_slab = slab_percentage * (0.5 * frame_span * frame_spacing) * slab_thickness * concrete_density

ex = pad_width_x / 2 - dist_from_edge_x_direction
ey = pad_width_y / 2 - dist_from_edge_y_direction

# use envelopes of forces
Rx = max(Rx_plus,Rx_minus)
Ry = max(Ry_plus_lngtnl_bracing, Ry_minus_lngtnl_bracing)
Rz = Rz_down
Mx = max(Mx_plus,Mx_minus)
My = max(My_plus,My_minus)
sigma_Rx = Rx * pad_depth / (pad_width_x**2 * pad_width_y / 6)
sigma_Ry = Ry * pad_depth / (pad_width_x * pad_width_y**2 / 6) 
sigma_Rz = Rz / (pad_width_x * pad_width_y)
sigma_Rz = sigma_Rz + Rz * ex / (pad_width_x**2 * pad_width_y / 6)
sigma_Rz = sigma_Rz + Rz * ey / (pad_width_x * pad_width_y**2 / 6)
sigma_Mx = Mx * dist_from_edge_y_direction / pad_width_y
sigma_My = My * dist_from_edge_x_direction / pad_width_x

stress = sigma_Rx + sigma_Ry + sigma_Rz + sigma_Mx + sigma_My

In [221]:
print("weight_pad                        = ", ("%2.2f" % weight_pad), " [kN]")
print("weight_slab                       = ", ("%2.2f" % weight_slab), " [kN]")
print("sigma_pad                         = ", ("%2.2f" % sigma_pad), " [kPa]")
print("sigma_Rx        = Rx * pad_depth / (pad_width_x**2 * pad_width_y / 6)")
print("sigma_Ry        = Ry * pad_depth / (pad_width_x * pad_width_y**2 / 6)") 
print("sigma_Rz        = Rz / (pad_width_x * pad_width_y)")
print("sigma_Rz        = sigma_Rz + Rz * ex / (pad_width_x**2 * pad_width_y / 6)")
print("sigma_Rz        = sigma_Rz + Rz * ey / (pad_width_x * pad_width_y**2 / 6)")
print("sigma_Mx        = Mx * dist_from_edge_y_direction / pad_width_y")
print("sigma_My        = My * dist_from_edge_x_direction / pad_width_x")
print("sigma_Rx                          = ", ("%2.2f" % sigma_Rx), " [kPa]")
print("sigma_Ry                          = ", ("%2.2f" % sigma_Ry), " [kPa]")
print("sigma_Rz                          = ", ("%2.2f" % sigma_Rz), " [kPa]")
print("sigma_Mx                          = ", ("%2.2f" % sigma_Mx), " [kPa]")
print("sigma_My                          = ", ("%2.2f" % sigma_My), " [kPa]")

print("stress                            = ", ("%2.2f" % stress), " [kPa]")

weight_pad                        =  24.00  [kN]
weight_slab                       =  88.20  [kN]
sigma_pad                         =  24.00  [kPa]
sigma_Rx        = Rx * pad_depth / (pad_width_x**2 * pad_width_y / 6)
sigma_Ry        = Ry * pad_depth / (pad_width_x * pad_width_y**2 / 6)
sigma_Rz        = Rz / (pad_width_x * pad_width_y)
sigma_Rz        = sigma_Rz + Rz * ex / (pad_width_x**2 * pad_width_y / 6)
sigma_Rz        = sigma_Rz + Rz * ey / (pad_width_x * pad_width_y**2 / 6)
sigma_Mx        = Mx * dist_from_edge_y_direction / pad_width_y
sigma_My        = My * dist_from_edge_x_direction / pad_width_x
sigma_Rx                          =  72.00  [kPa]
sigma_Ry                          =  120.00  [kPa]
sigma_Rz                          =  110.00  [kPa]
sigma_Mx                          =  0.00  [kPa]
sigma_My                          =  0.00  [kPa]
stress                            =  302.00  [kPa]


In [222]:

UF_stress = stress / Rd

if UF_stress > 1.0:
    fail_stress = True
    print("FAIL")
else:
    fail_stress = False
    print("PASS")

PASS


In [223]:
print("UF_stress                         = ", ("%2.2f" % UF_stress), " [-]")

if fail_stress == True:
    print ("Stress check FAILED. Utilization is " + str(round(100 * UF_stress,2)) + "%")
else:
    print ("Stress check PASSED. Utilization is " + str(round(100 * UF_stress,2)) + "%")

UF_stress                         =  0.95  [-]
Stress check PASSED. Utilization is 94.85%


### Check uplift

In [224]:
if slab_engaged_uplift == True:
    UF_uplift = (Rz_uplift + Rz_uplift_bracing) / (weight_pad + weight_slab) 
else:
    UF_uplift = (Rz_uplift + Rz_uplift_bracing) / weight_pad 
  
    
if UF_uplift > 1.0:
    fail_uplift = True
else:
    fail_uplift = False    

In [225]:
print("weight_pad                        = ", ("%2.2f" % weight_pad), " [kN]")
print("weight_slab                       = ", ("%2.2f" % weight_slab), " [kN]")
print("UF_uplift                         = ", ("%2.2f" % UF_uplift), " [-]")

if fail_uplift == True:
    print ("Uplift check FAILED. Utilization is " + str(round(100 * UF_uplift,2)) + "%")
else:
    print ("Uplift check PASSED. Utilization is " + str(round(100 * UF_uplift,2)) + "%")

weight_pad                        =  24.00  [kN]
weight_slab                       =  88.20  [kN]
UF_uplift                         =  0.49  [-]
Uplift check PASSED. Utilization is 49.02%


### Check slab reinforcement (if engaged)

In [226]:
if slab_engaged_rotation == True:
    fcd = slab_concrete_grade
    fy = slab_reinforcement_fy
    b_eff = frame_span / 4
    Na_top = fy * slab_reinforcement_area_top * pad_width_x
    Na_btm = fy * slab_reinforcement_area_btm * pad_width_x
    x_top = Na_top / 0.85 / fcd / b_eff
    x_btm = Na_btm / 0.85 / fcd / b_eff
    # lever arm
    z_hogging = slab_thickness - 0.5 * x_top - slab_reinf_dist_btm
    z_sagging = slab_thickness - 0.5 * x_btm - slab_reinf_dist_top
    M_Rd_slab_hogging = Na_btm * z_hogging
    M_Rd_slab_sagging = Na_top * z_sagging
    #
    #...now, let's assume envelope, may be conservative but avoids confusion
    M_Rd_slab = min(M_Rd_slab_hogging, M_Rd_slab_sagging)
    

In [227]:
if slab_engaged_rotation == True:
    print("fcd                               = ", ("%2.2f" % fcd), " [kPa]")
    print("b_eff                             = ", ("%2.2f" % b_eff), " [m]")
    print("Na_top                            = ", ("%2.2f" % Na_top), " [kN]")
    print("Na_btm                            = ", ("%2.2f" % Na_btm), " [kN]")
    print("x_top                             = ", ("%2.4f" % x_top), " [m]")
    print("x_btm                             = ", ("%2.4f" % x_btm), " [m]")
    print("z_hogging                         = ", ("%2.2f" % z_hogging), " [m]")
    print("z_sagging                         = ", ("%2.2f" % z_sagging), " [m]")
    print("M_Rd_slab_hogging                 = ", ("%2.2f" % M_Rd_slab_hogging), " [kNm]")
    print("M_Rd_slab_sagging                 = ", ("%2.2f" % M_Rd_slab_sagging), " [kNm]")
    print("M_Rd_slab                         = ", ("%2.2f" % M_Rd_slab), " [kNm]")
else:
    print("Slab is considered not to be rotationally engaged.")

Slab is considered not to be rotationally engaged.


In [228]:
# Check moment resistance of slab
# Assuming mesh reinforcement
if slab_engaged_rotation == True:
    M = max(My_plus, My_minus, Mx_plus, Mx_minus)
    UF_reinforcement = M / M_Rd_slab

    if UF_reinforcement > 1.0:
        fail_reinforcement = True
    else:
        fail_reinforcement = False    

In [229]:
if slab_engaged_rotation == True:
    print("UF_reinforcement                  = ", ("%2.2f" % UF_reinforcement), " [-]")
    if fail_reinforcement == True:
        print ("Reinforcement check FAILED. Utilization is " + str(round(100 * UF_reinforcement,2)) + "%")
    else:
        print ("Reinforcement check PASSED. Utilization is " + str(round(100 * UF_reinforcement,2)) + "%")